# Experiment: Fidelity of Iterative Projection vs. Initial State Complexity and Qubit Count

## Overview
This notebook investigates how the fidelity between an initial quantum state and the state resulting from an iterative projection process changes with:
1. The complexity (ansatz depth) of the initial state.
2. The total number of qubits in the system.

The process for each run is:
1. Generate an initial N-qubit quantum state, `|psi_initial>`, using a `RealAmplitudes` ansatz with a specific `PSI_ANSATZ_DEPTH` and `NUM_QUBITS_EXP`.
2. Optimize rank-1 projectors `P_ij*` for each nearest-neighbor pair to minimize `<psi_initial|P_ij*|psi_initial>`.
3. Sequentially apply filter operators `(I - P_ij*)` to `|psi_initial>` and renormalize, producing `|psi_final>`.
4. Calculate the fidelity $F(|\psi_{initial}\rangle, |\psi_{final}\rangle)$.

This is repeated for several random instances at each `PSI_ANSATZ_DEPTH` and for each `NUM_QUBITS_EXP`. The average fidelity for each depth and qubit count is then plotted, with separate lines for each qubit count showing fidelity vs. ansatz depth.

## Requirements
- `numpy`
- `qiskit`
- `scipy`
- `matplotlib` (for plotting)

## 1. Imports and Setup

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, TwoLocal
from qiskit.quantum_info import Statevector, Operator, SparsePauliOp, state_fidelity
from qiskit.visualization import circuit_drawer
import matplotlib.pyplot as plt # For plotting
import time # To time the experiment

try:
    from scipy.optimize import minimize
except ModuleNotFoundError:
    print("ERROR: scipy is not installed. Please install it: pip install scipy")
    def minimize(*args, **kwargs):
        raise RuntimeError("scipy.optimize.minimize not found. Please install scipy.")

## 2. Experiment Configuration

In [ ]:
# --- Experiment Configuration ---
QUBIT_COUNTS_EXP = [3, 4, 5, 6, 7, 8] # List of qubit numbers to test
MAX_PSI_ANSATZ_DEPTH = 8             # Max depth for RealAmplitudes (0 to MAX_PSI_ANSATZ_DEPTH)
NUM_RUNS_PER_CONFIG = 3            # Number of random instances per (qubit_count, depth) for averaging

# --- Fixed Parameters from previous setup (can be tuned) ---
PROJECTOR_U_REPS = 1       # Repetitions in TwoLocal for U(theta)
SCIPY_COBYLA_MAXITER = 1500  # Max iterations for COBYLA (reduced for broader experiment)
SCIPY_COBYLA_TOL = 1e-4      # Tolerance for COBYLA (adjusted for speed)
MAIN_SEED = 42               # Main seed for reproducibility of the entire experiment

# Set NumPy's random seed. This will be reset for each run for comparable randomness.
np.random.seed(MAIN_SEED)
print(f"Main NumPy random seed for experiment: {MAIN_SEED}")

VISUALIZE_CIRCUITS = False # Set to True to draw circuits (can be very verbose)
VERBOSE_OPTIMIZATION = False # Set to True for detailed optimizer output

## 3. Core Logic Function
This function encapsulates the process for a single experimental run (one qubit count, one depth, one random seed).

In [ ]:
def run_single_projection_experiment(num_qubits, psi_depth, run_seed):
    """Runs one instance of the state generation, projector optimization, 
    and iterative filtering, returning the final fidelity."""
    np.random.seed(run_seed) # Set seed for this specific run

    # --- Step 1: Create initial quantum state |psi_initial> ---
    psi_ansatz = RealAmplitudes(num_qubits, reps=psi_depth, entanglement='linear')
    num_psi_params = psi_ansatz.num_parameters
    # Ensure parameters are generated even if num_psi_params is 0 (for depth 0)
    if num_psi_params > 0:
        random_psi_params = np.random.rand(num_psi_params) * 2 * np.pi
        psi_circuit = psi_ansatz.assign_parameters(random_psi_params)
    else: # Depth 0 means no parameters, just the |0...0> state implicitly
        psi_circuit = QuantumCircuit(num_qubits) # Effectively |0...0>

    # Visualization (optional and conditional)
    # if VISUALIZE_CIRCUITS and psi_depth > 0: ... 

    try:
        psi_initial_vector = Statevector(psi_circuit)
    except Exception as e:
        # print(f"    ERROR (Run {run_seed}, Depth {psi_depth}, Qubits {num_qubits}): Could not create Statevector: {e}")
        return np.nan # Return NaN on failure

    # --- Step 2: Define base projectors and U(theta) ansatz ---
    proj_00_on_qubits_static_pauli_sum = {}
    if num_qubits > 1: # Projectors only make sense for > 1 qubit
        for q1_idx in range(num_qubits - 1):
            q2_idx = q1_idx + 1
            pauli_list = [
                ("I" * num_qubits, 0.25),
                ("".join(['Z' if i == q1_idx else 'I' for i in range(num_qubits)]), 0.25),
                ("".join(['Z' if i == q2_idx else 'I' for i in range(num_qubits)]), 0.25),
                ("".join(['Z' if i == q1_idx or i == q2_idx else 'I' for i in range(num_qubits)]), 0.25)
            ]
            reversed_pauli_list = [(p_str[::-1], coeff) for p_str, coeff in pauli_list]
            proj_00_on_qubits_static_pauli_sum[(q1_idx, q2_idx)] = SparsePauliOp.from_list(reversed_pauli_list)

    u_ansatz_2q = TwoLocal(2, rotation_blocks=['ry', 'rz'], entanglement_blocks='cx',
                           reps=PROJECTOR_U_REPS, entanglement='linear')
    num_u_params = u_ansatz_2q.num_parameters

    # --- Step 3: Optimize projectors ---
    optimized_projectors_P_star = []
    if num_qubits > 1:
        qubit_pairs = [(i, i + 1) for i in range(num_qubits - 1)]
        for q1, q2 in qubit_pairs:
            base_proj_spo = proj_00_on_qubits_static_pauli_sum[(q1, q2)]
            def cost_func(params):
                try:
                    u_circ_2q = u_ansatz_2q.assign_parameters(params)
                    u_full_qc_ = QuantumCircuit(num_qubits)
                    u_full_qc_.compose(u_circ_2q, qubits=[q1, q2], inplace=True)
                    u_op = Operator(u_full_qc_)
                    p_op_obj = u_op @ base_proj_spo @ u_op.adjoint()
                    return np.real(psi_initial_vector.expectation_value(p_op_obj))
                except Exception:
                    return np.nan 
            
            init_params = np.random.rand(num_u_params) * 2 * np.pi
            try:
                opt_res = minimize(fun=cost_func, x0=init_params, method='COBYLA',
                                   tol=SCIPY_COBYLA_TOL, 
                                   options={'maxiter': SCIPY_COBYLA_MAXITER, 'disp': VERBOSE_OPTIMIZATION})
                if opt_res.success or (not np.isnan(opt_res.fun) and opt_res.fun < 1.0): # Accept if fun is reasonable
                    u_circ_2q_opt_ = u_ansatz_2q.assign_parameters(opt_res.x)
                    u_full_qc_opt_ = QuantumCircuit(num_qubits)
                    u_full_qc_opt_.compose(u_circ_2q_opt_, qubits=[q1, q2], inplace=True)
                    u_op_opt_ = Operator(u_full_qc_opt_)
                    p_star_obj = u_op_opt_ @ base_proj_spo @ u_op_opt_.adjoint()
                    optimized_projectors_P_star.append({'qubits': (q1,q2), 'P_star_op': p_star_obj})
            except Exception:
                pass 

    # --- Step 4: Iterative projection ---
    if not optimized_projectors_P_star and num_qubits > 1:
        # If optimization failed for all projectors but there were pairs to optimize
        # this run might not be very informative, could return NaN or treat as fidelity 1
        # For now, assume if no projectors, state doesn't change.
        pass 
        
    current_psi = psi_initial_vector.copy()
    if num_qubits > 0 : # Identity op only if there are qubits
        id_op = Operator(np.eye(2**num_qubits, dtype=complex))
    else: # Should not happen with QUBIT_COUNTS_EXP setting
        return 1.0 

    for p_info in optimized_projectors_P_star:
        k_op = id_op - p_info['P_star_op']
        proj_psi_unnorm = current_psi.evolve(k_op)
        norm = np.linalg.norm(proj_psi_unnorm.data)
        if norm < 1e-9:
            current_psi = Statevector(proj_psi_unnorm.data, dims=proj_psi_unnorm.dims())
            break
        current_psi = Statevector(proj_psi_unnorm.data / norm, dims=proj_psi_unnorm.dims())
    
    final_psi = current_psi

    # --- Step 5: Calculate Fidelity ---
    final_norm = np.linalg.norm(final_psi.data)
    if final_norm < 1e-9:
        return 0.0 
    if abs(final_norm - 1.0) > 1e-7:
        final_psi = Statevector(final_psi.data / final_norm, dims=final_psi.dims())
        
    init_norm = np.linalg.norm(psi_initial_vector.data)
    if abs(init_norm - 1.0) > 1e-7:
        # This shouldn't happen if Statevector(circuit) works correctly
        psi_initial_vector = Statevector(psi_initial_vector.data / init_norm, dims=psi_initial_vector.dims())
        
    return state_fidelity(psi_initial_vector, final_psi)



## 4. Running the Experiment
Outer loop iterates through `QUBIT_COUNTS_EXP`. Inner loop iterates through `PSI_ANSATZ_DEPTH`. For each configuration, `NUM_RUNS_PER_CONFIG` are performed.

In [ ]:
depth_values = list(range(MAX_PSI_ANSATZ_DEPTH + 1))
results_by_qubit_count = {} # Store {num_qubits: {'avg_fidelities': [], 'std_fidelities': []}}

overall_experiment_start_time = time.time()

for n_qubits in QUBIT_COUNTS_EXP:
    print(f"\n===== Processing for NUM_QUBITS_EXP = {n_qubits} =====")
    current_qubit_avg_fidelities = []
    current_qubit_std_fidelities = []
    
    for depth in depth_values:
        print(f"  Processing PSI_ANSATZ_DEPTH = {depth} for {n_qubits} qubits...")
        current_config_fidelities = []
        config_start_time = time.time()
        
        for i_run in range(NUM_RUNS_PER_CONFIG):
            run_specific_seed = MAIN_SEED + n_qubits * 100 + depth * 10 + i_run # Ensure unique seeds
            # print(f"    Run {i_run + 1}/{NUM_RUNS_PER_CONFIG} (Seed: {run_specific_seed})...") # Verbose
            
            fidelity = run_single_projection_experiment(n_qubits, depth, run_specific_seed)
            
            if not np.isnan(fidelity):
                current_config_fidelities.append(fidelity)
            # else: print(f"      Run failed for seed {run_specific_seed}, fidelity is NaN.") # Verbose
                
        config_end_time = time.time()
        # print(f"    Finished all runs for depth {depth} in {config_end_time - config_start_time:.2f}s") # Verbose

        if current_config_fidelities:
            avg_fid = np.mean(current_config_fidelities)
            std_fid = np.std(current_config_fidelities)
            current_qubit_avg_fidelities.append(avg_fid)
            current_qubit_std_fidelities.append(std_fid)
            print(f"    Avg Fidelity for {n_qubits} qubits, depth {depth}: {avg_fid:.4f} +/- {std_fid:.4f}")
        else:
            current_qubit_avg_fidelities.append(np.nan)
            current_qubit_std_fidelities.append(np.nan)
            print(f"    No successful runs for {n_qubits} qubits, depth {depth}. Avg Fidelity: NaN")
    
    results_by_qubit_count[n_qubits] = {
        'avg_fidelities': current_qubit_avg_fidelities,
        'std_fidelities': current_qubit_std_fidelities
    }
    print(f"  --- Finished processing for {n_qubits} qubits ---")

overall_experiment_end_time = time.time()
print(f"\nTotal experiment time: {overall_experiment_end_time - overall_experiment_start_time:.2f} seconds")

## 5. Plotting Results
Plot the average fidelity against the `PSI_ANSATZ_DEPTH` for each number of qubits, with distinct lines.

In [ ]:
plt.figure(figsize=(12, 8))

for n_qubits, results in results_by_qubit_count.items():
    # Filter out NaN values for plotting if a whole (depth, qubit_count) config failed
    valid_indices = ~np.isnan(results['avg_fidelities'])
    plot_depths = np.array(depth_values)[valid_indices]
    plot_avg_fidelities = np.array(results['avg_fidelities'])[valid_indices]
    plot_std_fidelities = np.array(results['std_fidelities'])[valid_indices]
    
    if len(plot_depths) > 0: # Only plot if there's data
        plt.errorbar(plot_depths, plot_avg_fidelities, yerr=plot_std_fidelities, 
                     marker='o', linestyle='-', capsize=3, label=f'{n_qubits} Qubits')

plt.xlabel("Initial State Ansatz Depth (PSI_ANSATZ_DEPTH)")
plt.ylabel("Average Fidelity (F(|psi_initial>, |psi_final>))")
plt.title(f"Avg Fidelity vs. State Complexity for Different Qubit Counts ({NUM_RUNS_PER_CONFIG} Runs/Config)")
plt.xticks(depth_values)
plt.ylim(-0.05, 1.05) # Fidelity is between 0 and 1
plt.grid(True, which='both', linestyle='--')
plt.legend(title="Number of Qubits")
plt.show()

## 6. Execution Guidance & Expected Output

1.  **Kernel:** Ensure a Python 3 kernel with `qiskit`, `numpy`, `scipy`, and `matplotlib`.
2.  **Run All Cells:** Execute cells sequentially from top to bottom.
3.  **Configuration (Cell 2):**
    *   `QUBIT_COUNTS_EXP`: List of qubit numbers (e.g., `[3, 4, 5, 6, 7, 8]`).
    *   `MAX_PSI_ANSATZ_DEPTH`: Max depth for the `RealAmplitudes` ansatz.
    *   `NUM_RUNS_PER_CONFIG`: Number of random instances per (qubit count, depth) pair. Increase for smoother averages, decrease for faster runs.
    *   `SCIPY_COBYLA_MAXITER`, `SCIPY_COBYLA_TOL`: Optimizer settings. These are critical for runtime. For an exploratory run, `MAXITER` might be lowered (e.g., 500-1500) and `TOL` increased (e.g., 1e-4) to speed things up, especially as the number of qubits increases.
4.  **Patience:** This experiment will be **significantly more computationally intensive** than the single run due to the nested loops over qubit counts, ansatz depths, and random runs. The total number of optimizations is `len(QUBIT_COUNTS_EXP) * (MAX_PSI_ANSATZ_DEPTH + 1) * NUM_RUNS_PER_CONFIG * (avg_num_projectors_per_qubit_count)`.
    *   For example, with `QUBIT_COUNTS_EXP = [3,4,5,6,7,8]` (6 settings), `MAX_PSI_ANSATZ_DEPTH = 8` (9 depths), `NUM_RUNS_PER_CONFIG = 3`, for 8 qubits (7 projectors): `6 * 9 * 3 * ~7` optimizations could be a rough lower bound for the 8-qubit case alone if all other qubit counts are fast. Consider reducing `NUM_RUNS_PER_CONFIG` to 1 or 2, or `MAX_PSI_ANSATZ_DEPTH` to a smaller value (e.g., 4 or 5) for initial tests.

### Expected Output
- Print statements showing the progress for each qubit count, depth, and run.
- The average fidelity (and standard deviation) for each (qubit count, depth) configuration.
- A final plot showing Average Fidelity on the Y-axis and Initial State Ansatz Depth on the X-axis, with distinct lines for each number of qubits tested.